In [1]:
# Editable install for development
# In real use this should be "pip install o11y"
!pip install ./dist/*.whl --force-reinstall

Processing ./dist/o11y-0.1.0-py3-none-any.whl
  Using cached python_logging_loki-0.3.1-py3-none-any.whl.metadata (3.4 kB)
  Using cached requests-2.31.0-py3-none-any.whl.metadata (4.6 kB)
  Using cached rfc3339-6.2-py3-none-any.whl.metadata (816 bytes)
  Using cached charset_normalizer-3.3.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (33 kB)
  Using cached idna-3.7-py3-none-any.whl.metadata (9.9 kB)
  Using cached urllib3-2.2.1-py3-none-any.whl.metadata (6.4 kB)
  Using cached certifi-2024.2.2-py3-none-any.whl.metadata (2.2 kB)
Using cached python_logging_loki-0.3.1-py3-none-any.whl (7.0 kB)
Using cached requests-2.31.0-py3-none-any.whl (62 kB)
Using cached certifi-2024.2.2-py3-none-any.whl (163 kB)
Using cached charset_normalizer-3.3.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (140 kB)
Using cached idna-3.7-py3-none-any.whl (66 kB)
Using cached rfc3339-6.2-py3-none-any.whl (5.5 kB)
Using cached urllib3-2.2.1-py3-none-any.whl (121 kB)
  Attempt

In [2]:
import o11y

In [3]:
import os
# Verify we have credentials set in environment
# In principle it should be possible to do "o11y login" to set this from the command line
print(os.environ['GF_AI_TRAINING_CREDS'])

83bcaff6228b39bbe431af5e19fb4368e2a03dd3@localhost:8000


In [10]:
import random 

epochs = 10
lr = 0.01

run = o11y.init(
    # Set the project where this run will be logged
    project="my-awesome-project",
    # Track hyperparameters and run metadata
    metadata={
        "config": {
            "learning_rate": lr,
            "epochs": epochs,
            "feed_forward": {
                "activation": "swiglu",
                "d_model": 1024,
            }
        },
        "git_commit": "some_hash_goes_here",
    },
)

offset = random.random() / 5
print(f"lr: {lr}")

# simulating a training run
for epoch in range(2, epochs):
    acc = 1 - 2**-epoch - random.random() / epoch - offset
    loss = 2**-epoch + random.random() / epoch + offset
    print(f"epoch={epoch}, accuracy={acc}, loss={loss}")
    o11y.log({"accuracy": acc, "loss": loss})
    if epoch == epochs:
        o11y.update({
            "final_accuracy": acc,
            "final_loss": loss
        })

--- Logging error ---
Traceback (most recent call last):
  File "/opt/conda/lib/python3.11/site-packages/logging_loki/handlers.py", line 81, in emit
    self.emitter(record, self.format(record))
  File "/opt/conda/lib/python3.11/site-packages/logging_loki/emitter.py", line 57, in __call__
    raise ValueError("Unexpected Loki API response status code: {0}".format(resp.status_code))
ValueError: Unexpected Loki API response status code: 200
Call stack:
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/opt/conda/lib/python3.11/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/opt/conda/lib/python3.11/site-packages/traitlets/config/application.py", line 1075, in launch_instance
    app.start()
  File "/opt/conda/lib/python3.11/site-packages/ipykernel/kernelapp.py", line 739, in start
    self.io_loop.start()
  File "/opt/conda/lib/python3.11/site-packages/tornado/platform/asyncio

lr: 0.01
epoch=2, accuracy=0.48037282168706974, loss=0.4611996132758005
epoch=3, accuracy=0.8327567010711399, loss=0.37886422727708424
epoch=4, accuracy=0.8378506809335033, loss=0.2747658631788615
epoch=5, accuracy=0.9056050202833931, loss=0.14527804194072716
epoch=6, accuracy=0.8774964140194966, loss=0.1342930881779633
epoch=7, accuracy=0.9641874643956766, loss=0.04483947011121521
epoch=8, accuracy=0.9021568545306147, loss=0.14375248634661233
epoch=9, accuracy=0.9267603225036217, loss=0.1285421792189219
